# 빅데이터 분석기사 실기 준비 놀이터
- https://www.kaggle.com/datasets/agileteam/bigdatacertificationkr

## T2-1. Titanic (Classification) / 타이타닉

### 1. 생존여부 예측모델 만들기
- 데이터셋 : titanic_train.csv
    - y_train: 생존여부(학습용)
    - trian, X_test : 승객 정보 (학습용 및 평가용)
- 학습용 데이터 (X_train, y_train)을 이용하여 생존 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측값을 다음과 같은 형식의 CSV파일로 생성    
- https://www.kaggle.com/code/agileteam/t2-1-titanic-simple-baseline/notebook

#### 0. 시험 환경 세팅

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("./data/titanic_train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Survived', id_name='PassengerId')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 11), (179, 11), (712, 2), (179, 2))

#### 1. 라이브러리 및 데이터 호출

In [2]:
import numpy as np
import pandas as pd

X_train.shape, X_test.shape, y_train.shape

((712, 11), (179, 11), (712, 2))

#### 2. EDA

In [3]:
display(X_train.info(), X_train.head(), X_train.isnull().sum())
print('='*100)
display(y_train.info(), y_train.head(), y_train.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 90 to 116
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Pclass       712 non-null    int64  
 2   Name         712 non-null    object 
 3   Sex          712 non-null    object 
 4   Age          575 non-null    float64
 5   SibSp        712 non-null    int64  
 6   Parch        712 non-null    int64  
 7   Ticket       712 non-null    object 
 8   Fare         712 non-null    float64
 9   Cabin        170 non-null    object 
 10  Embarked     711 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 66.8+ KB


None

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
90,91,3,"Christmann, Mr. Emil",male,29.0,0,0,343276,8.0500,NaN,S
103,104,3,"Johansson, Mr. Gustaf Joel",male,33.0,0,0,7540,8.6542,NaN,S
577,578,1,"Silvey, Mrs. William Baird (Alice Munger)",female,39.0,1,0,13507,55.9000,E44,S
215,216,1,"Newell, Miss. Madeleine",female,31.0,1,0,35273,113.2750,D36,C
191,192,2,"Carbines, Mr. William",male,19.0,0,0,28424,13.0000,NaN,S


PassengerId      0
Pclass           0
Name             0
Sex              0
Age            137
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          542
Embarked         1
dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 90 to 116
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  712 non-null    int64
 1   Survived     712 non-null    int64
dtypes: int64(2)
memory usage: 16.7 KB


None

,PassengerId,Survived
90,91,0
103,104,0
577,578,1
215,216,1
191,192,0


PassengerId    0
Survived       0
dtype: int64

In [4]:
for c in X_train.columns:
    print(c)
    print(X_train[c].unique())
    print('='*100)

PassengerId
[ 91 104 578 216 192 184 571 439 105 867 634 175 891  74 311 465 229 791
 102 300 440   2  59 843 701 214 536 866 496 889 856 722  61 717 521 739
 626 243 723 509 151 333 513  77 785 569 382 695 845   7 852 506 329 516
 654 570 317  35 703 231 198 466 619 640 697 817 630 824 492 887 651 736
 148  44 876 208   4 152 163 518  81 732 242 354 125 449 868 808 772 726
  53 884 345 451 338  15 527 228 515 577 320 499 428 401 765 348 790 270
 244 371 655 472 807 617 752 378 119 461 545 218 768  11 684 535 355 251
  14 848 445 159 485 316 467 734 195 514  84 126  90 424 116 552 764 477
 650 326 688 828 858 685 259  87  12 434  25 356 226 341 254 253 249 265
 846 123 721 602 754 789 462 511 550 805 470  54 446 205   1 762 591 340
 344 769 293 750 310 227 403  50 780 603 799 299 286  89  99  31 315 727
  78  36 463 628 146 522 559 794 375 328 725 183 114 711  79   3 245 409
 294 572 305  13 682 455 663  95 150 557 395 859 589 854 361 837 200 528
 607 111 352  52 844 565 758 353 728 78

#### 3. Preprocessing

> 수치형 컬럼
- PassengerId : 불필요컬럼 제거
- Pclass : 그대로
- Age : 결측치 중앙값 대체
- SibSp, Parch : 그대로
- Fare : 그대로

In [5]:
int_col = X_train.select_dtypes(exclude='object').columns

for c in int_col:
    print(c, X_train[c].nunique())
    print(np.sort(X_train[c].unique()))
    print('='*100)

PassengerId 712
[  1   2   3   4   5   6   7   8  11  12  13  14  15  16  18  19  20  21
  23  24  25  26  27  28  30  31  32  34  35  36  39  40  43  44  45  48
  50  51  52  53  54  55  56  57  58  59  60  61  64  65  66  68  69  70
  71  72  73  74  76  77  78  79  81  82  83  84  85  86  87  88  89  90
  91  92  94  95  97  98  99 101 102 103 104 105 106 108 109 110 111 113
 114 115 116 117 118 119 120 122 123 124 125 126 127 128 129 130 132 136
 137 138 139 140 141 143 144 145 146 147 148 149 150 151 152 153 154 155
 156 157 159 161 162 163 164 165 166 167 169 170 171 172 173 174 175 176
 177 179 181 182 183 184 185 186 189 190 191 192 193 194 195 196 197 198
 199 200 203 204 205 207 208 212 214 215 216 218 219 222 224 226 227 228
 229 230 231 234 235 237 239 240 241 242 243 244 245 246 247 249 250 251
 252 253 254 255 256 258 259 260 262 263 264 265 266 267 269 270 271 272
 273 274 275 276 277 278 279 280 281 282 284 286 289 291 292 293 294 295
 296 297 298 299 300 301 305 306 30

In [6]:
int_col = int_col.drop('PassengerId')

print(X_train[int_col].isnull().sum(), '\n')
print(X_test[int_col].isnull().sum())

Pclass      0
Age       137
SibSp       0
Parch       0
Fare        0
dtype: int64 

Pclass     0
Age       40
SibSp      0
Parch      0
Fare       0
dtype: int64


In [7]:
X_train['Age'] = X_train['Age'].fillna(round(X_train['Age'].mean(),0))
X_test['Age'] = X_test['Age'].fillna(round(X_test['Age'].mean(),0))

print(X_train[int_col].isnull().sum(), '\n')
print(X_test[int_col].isnull().sum())

Pclass    0
Age       0
SibSp     0
Parch     0
Fare      0
dtype: int64 

Pclass    0
Age       0
SibSp     0
Parch     0
Fare      0
dtype: int64


> 범주형 컬럼
- Name : 고유값 개수가 각 행마다 다르므로 제외
- Sex : 인코딩 필요
- Ticket : 고유값 개수가 너무 많으므로 제외
- Cabin : 결측치가 많으므로 제외
- Embarked : 결측치 최빈값으로 대체, 인코딩 필요

In [8]:
obj_col = X_train.select_dtypes(include='object').columns

for c in obj_col:
    print(c, X_train[c].nunique())
    print(X_train[c].unique())
    print('='*100)

Name 712
['Christmann, Mr. Emil' 'Johansson, Mr. Gustaf Joel'
 'Silvey, Mrs. William Baird (Alice Munger)' 'Newell, Miss. Madeleine'
 'Carbines, Mr. William' 'Becker, Master. Richard F' 'Harris, Mr. George'
 'Fortune, Mr. Mark' 'Gustafsson, Mr. Anders Vilhelm'
 'Duran y More, Miss. Asuncion' 'Parr, Mr. William Henry Marsh'
 'Smith, Mr. James Clinch' 'Dooley, Mr. Patrick'
 'Chronopoulos, Mr. Apostolos' 'Hays, Miss. Margaret Bechstein'
 'Maisner, Mr. Simon' 'Fahlstrom, Mr. Arne Jonas'
 'Keane, Mr. Andrew "Andy"' 'Petroff, Mr. Pastcho ("Pentcho")'
 'Baxter, Mrs. James (Helene DeLaudeniere Chaput)'
 'Kvillner, Mr. Johan Henrik Johannesson'
 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)'
 'West, Miss. Constance Mirium' 'Serepeca, Miss. Augusta'
 'Astor, Mrs. John Jacob (Madeleine Talmadge Force)'
 'Givard, Mr. Hans Kristensen' 'Hart, Miss. Eva Miriam'
 'Bystrom, Mrs. (Karolina)' 'Yousseff, Mr. Gerious'
 'Johnston, Miss. Catherine Helen "Carrie"' 'Aks, Mrs. Sam (Leah Rosen)'
 'Jensen,

In [9]:
print(X_train[obj_col].isnull().sum(), '\n')
print(X_test[obj_col].isnull().sum())

Name          0
Sex           0
Ticket        0
Cabin       542
Embarked      1
dtype: int64 

Name          0
Sex           0
Ticket        0
Cabin       145
Embarked      1
dtype: int64


In [10]:
obj_col = obj_col.drop(['Name', 'Ticket','Cabin'])

In [11]:
X_train['Embarked'] = X_train['Embarked'].fillna(X_train['Embarked'].mode()[0])
X_test['Embarked'] = X_test['Embarked'].fillna(X_test['Embarked'].mode()[0])

print(X_train[obj_col].isnull().sum(), '\n')
print(X_test[obj_col].isnull().sum())

Sex         0
Embarked    0
dtype: int64 

Sex         0
Embarked    0
dtype: int64


In [12]:
print(X_train[obj_col].shape, X_train[int_col].shape)
print(X_test[obj_col].shape, X_test[int_col].shape)

(712, 2) (712, 5)
(179, 2) (179, 5)


In [13]:
X = pd.concat([X_train[obj_col], X_train[int_col]], axis=1)
test = pd.concat([X_test[obj_col], X_test[int_col]], axis=1)
y = y_train['Survived']

display(X, test, y)

,Sex,Embarked,Pclass,Age,SibSp,Parch,Fare
90,male,S,3,29.0,0,0,8.0500
103,male,S,3,33.0,0,0,8.6542
577,female,S,1,39.0,1,0,55.9000
215,female,C,1,31.0,1,0,113.2750
191,male,S,2,19.0,0,0,13.0000
...,...,...,...,...,...,...,...
621,male,S,1,42.0,1,0,52.5542
128,female,C,3,29.0,1,1,22.3583
57,male,C,3,28.5,0,0,7.2292
341,female,S,1,24.0,3,2,263.0000


,Sex,Embarked,Pclass,Age,SibSp,Parch,Fare
210,male,S,3,24.0,0,0,7.0500
876,male,S,3,20.0,0,0,9.8458
666,male,S,2,25.0,0,0,13.0000
819,male,S,3,10.0,3,2,27.9000
736,female,S,3,48.0,1,3,34.3750
...,...,...,...,...,...,...,...
494,male,S,3,21.0,0,0,8.0500
871,female,S,1,47.0,1,1,52.5542
530,female,S,2,2.0,1,1,26.0000
157,male,S,3,30.0,0,0,8.0500


90     0
103    0
577    1
215    1
191    0
      ..
621    1
128    1
57     0
341    1
116    0
Name: Survived, Length: 712, dtype: int64

In [14]:
X = pd.get_dummies(X)
test = pd.get_dummies(test)

display(X, test)

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
90,3,29.0,0,0,8.0500,0,1,0,0,1
103,3,33.0,0,0,8.6542,0,1,0,0,1
577,1,39.0,1,0,55.9000,1,0,0,0,1
215,1,31.0,1,0,113.2750,1,0,1,0,0
191,2,19.0,0,0,13.0000,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
621,1,42.0,1,0,52.5542,0,1,0,0,1
128,3,29.0,1,1,22.3583,1,0,1,0,0
57,3,28.5,0,0,7.2292,0,1,1,0,0
341,1,24.0,3,2,263.0000,1,0,0,0,1


,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
210,3,24.0,0,0,7.0500,0,1,0,0,1
876,3,20.0,0,0,9.8458,0,1,0,0,1
666,2,25.0,0,0,13.0000,0,1,0,0,1
819,3,10.0,3,2,27.9000,0,1,0,0,1
736,3,48.0,1,3,34.3750,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
494,3,21.0,0,0,8.0500,0,1,0,0,1
871,1,47.0,1,1,52.5542,1,0,0,0,1
530,2,2.0,1,1,26.0000,1,0,0,0,1
157,3,30.0,0,0,8.0500,0,1,0,0,1


In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = scaler.fit_transform(X)
test = scaler.transform(test)

X, test

(array([[ 0.84874956, -0.02556909, -0.48584612, ..., -0.49692575,
         -0.29217435,  0.61848547],
        [ 0.84874956,  0.27975904, -0.48584612, ..., -0.49692575,
         -0.29217435,  0.61848547],
        [-1.52574822,  0.73775123,  0.42447608, ..., -0.49692575,
         -0.29217435,  0.61848547],
        ...,
        [ 0.84874956, -0.0637351 , -0.48584612, ...,  2.01237307,
         -0.29217435, -1.61685287],
        [-1.52574822, -0.40722924,  2.24512048, ..., -0.49692575,
         -0.29217435,  0.61848547],
        [ 0.84874956,  3.14221021, -0.48584612, ..., -0.49692575,
          3.42261387, -1.61685287]]),
 array([[ 0.84874956, -0.40722924, -0.48584612, ..., -0.49692575,
         -0.29217435,  0.61848547],
        [ 0.84874956, -0.71255737, -0.48584612, ..., -0.49692575,
         -0.29217435,  0.61848547],
        [-0.33849933, -0.33089721, -0.48584612, ..., -0.49692575,
         -0.29217435,  0.61848547],
        ...,
        [-0.33849933, -2.08653393,  0.42447608, ..., -

#### 4. Modeling

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRFClassifier

from sklearn.metrics import roc_auc_score, accuracy_score

In [17]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=156)

In [18]:
lr = LogisticRegression(random_state=156)
rf = RandomForestClassifier(random_state=156)
gb = GradientBoostingClassifier(random_state=156)
xgb = XGBClassifier(random_state=156)
xgb_rf = XGBRFClassifier(random_state=156)

In [19]:
models = [lr, rf, gb, xgb, xgb_rf]

for model in models:
    model.fit(X_tr, y_tr)
    pred = model.predict(X_val)
    pred_proba = model.predict_proba(X_val)[:,1]
    
    name = model.__class__.__name__
    print(name)
    print('accuracy :', round(accuracy_score(y_val, pred),4))
    print('auc : ', round(roc_auc_score(y_val, pred_proba),4))
    print('='*100)

LogisticRegression
accuracy : 0.8042
auc :  0.8955
RandomForestClassifier
accuracy : 0.8392
auc :  0.9056
GradientBoostingClassifier
accuracy : 0.8462
auc :  0.882
XGBClassifier
accuracy : 0.8601
auc :  0.9088
XGBRFClassifier
accuracy : 0.8462
auc :  0.9121


In [20]:
xgb.fit(X, y)
pred = xgb.predict(test)

In [21]:
model.score(test, y_test['Survived']) # 가채점

0.770949720670391

In [22]:
outdata = pd.DataFrame({'PassengerId': X_test.PassengerId, 'Survived': pred})
outdata

,PassengerId,Survived
210,211,0
876,877,0
666,667,0
819,820,0
736,737,1
...,...,...
494,495,0
871,872,1
530,531,1
157,158,0


In [23]:
# outdata.to_csv('1234567.csv', index=False)